In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import BertTokenizer
from nltk.util import ngrams
import spacy
import re

In [2]:
# Load Data
train = pd.read_csv('../public_data/train/track_a/eng.csv')
val = pd.read_csv('../public_data/dev/track_a/eng_a.csv')
emotions = ['Joy', 'Sadness', 'Surprise', 'Fear', 'Anger']
emolex_path = "../EmoLex/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

In [3]:
# Load EmoLex Lexicon
def load_emolex(emolex_path):
    emolex = pd.read_csv(emolex_path, sep='\t', header=None, names=["Word", "Emotion", "Association"])
    emotion_dict = {}
    for _, row in emolex.iterrows():
        if row["Association"] == 1:
            word = row["Word"]
            emotion = row["Emotion"]
            if word not in emotion_dict:
                emotion_dict[word] = []
            emotion_dict[word].append(emotion)
    return emotion_dict

emotion_dict = load_emolex(emolex_path)

# Preprocessing Config
config = {'sep_pn': True, 'rm_pn': False, 'apply_lemmatization': True, 'apply_stemming': True, 'add_bigrams': True, 'rm_sw': False}

# Preprocessing Functions
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
nlp = spacy.load("en_core_web_sm")

In [4]:
def pre_process(text, config, target_emotion=None, emotion_dict=None):
    def separate_punctuation(text):
        text = re.sub(r"(\w)([.,;:!?\'\"”\)])", r"\1 \2", text)
        text = re.sub(r"([.,;:!?\'\"“\(\)])(\w)", r"\1 \2", text)
        return text

    def remove_punctuation(text):
        text = re.sub(r"[.,;:!?\'\"“”\(\)]", "", text)
        return text

    def tokenize_text(text):
        encoded_input = tokenizer(text, return_tensors='pt', add_special_tokens=True)
        tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
        return tokens

    def apply_stemming(tokens):
        stemmer = PorterStemmer()
        return [stemmer.stem(token) for token in tokens]

    def apply_lemmatization(tokens):
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(token) for token in tokens]

    def generate_ngrams_from_tokens(tokens, n):
        return [" ".join(gram) for gram in ngrams(tokens, n)]

    # Apply config options
    if config['sep_pn'] and not config['rm_pn']:
        text = separate_punctuation(text)
    if config['rm_pn'] and not config['sep_pn']:
        text = remove_punctuation(text)

    tokens = tokenize_text(text)
    if config['apply_stemming']:
        tokens = apply_stemming(tokens)
    if config['apply_lemmatization']:
        tokens = apply_lemmatization(tokens)
    if config['add_bigrams']:
        tokens += generate_ngrams_from_tokens(tokens, 2)
    if config['rm_sw']:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word.lower() not in stop_words]

    processed_text = " ".join(tokens)

    if target_emotion and emotion_dict:
        relevant_keywords = [word for word in tokens if target_emotion in emotion_dict.get(word, [])]
        if relevant_keywords:
            processed_text += f" [SEP] {' '.join(relevant_keywords)}"
        else:
            processed_text += " [SEP]"

    return processed_text

# Preprocess and Extract Features
vectorizer = CountVectorizer()

def preprocess_dataset_with_emotions(dataset, emotions, config, emotion_dict):
    augmented_data = {}
    for emotion in emotions:
        augmented_data[emotion] = [
            pre_process(text, config, target_emotion=emotion, emotion_dict=emotion_dict)
            for text in dataset
        ]
    return augmented_data

train_augmented = preprocess_dataset_with_emotions(train["text"], emotions, config, emotion_dict)
val_augmented = preprocess_dataset_with_emotions(val["text"], emotions, config, emotion_dict)

X_train = {emotion: vectorizer.fit_transform(train_augmented[emotion]).toarray() for emotion in emotions}
X_val = {emotion: vectorizer.transform(val_augmented[emotion]).toarray() for emotion in emotions}

# POS Tagging
def extract_pos_tags(texts):
    return [[token.pos_ for token in nlp(text)] for text in texts]

train_pos_tags = extract_pos_tags(train["text"])
val_pos_tags = extract_pos_tags(val["text"])

# POS Encoding
max_length = max(max(len(tags) for tags in train_pos_tags), max(len(tags) for tags in val_pos_tags))
train_pos_tags = [tags + ['PAD'] * (max_length - len(tags)) for tags in train_pos_tags]
val_pos_tags = [tags + ['PAD'] * (max_length - len(tags)) for tags in val_pos_tags]

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
train_pos_encoded = encoder.fit_transform(train_pos_tags)
val_pos_encoded = encoder.transform(val_pos_tags)

# Combine Features
combined_features = {
    emotion: np.concatenate((X_train[emotion], train_pos_encoded), axis=1) for emotion in emotions
}
validation_combined_features = {
    emotion: np.concatenate((X_val[emotion], val_pos_encoded), axis=1) for emotion in emotions
}

# Logistic Regression for Enhanced Features
y_train = train[emotions].values
lr_models = {}
lr_features = {}
val_lr_features = {}

for emotion in emotions:
    lr = LogisticRegression(max_iter=1000)
    lr.fit(combined_features[emotion], y_train[:, emotions.index(emotion)])
    lr_models[emotion] = lr
    lr_features[emotion] = lr.predict_proba(combined_features[emotion])
    val_lr_features[emotion] = lr.predict_proba(validation_combined_features[emotion])

final_train_features = {
    emotion: np.concatenate((combined_features[emotion], lr_features[emotion]), axis=1) for emotion in emotions
}
final_val_features = {
    emotion: np.concatenate((validation_combined_features[emotion], val_lr_features[emotion]), axis=1) for emotion in emotions
}

In [5]:
# Neural Network
model = nn.Sequential(
    nn.Linear(final_train_features[emotions[0]].shape[1], 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)

In [6]:
# DataLoader
for emotion in emotions:
    features_tensor = torch.tensor(final_train_features[emotion], dtype=torch.float32)
    labels_tensor = torch.tensor(y_train[:, emotions.index(emotion)], dtype=torch.float32).unsqueeze(1)
    dataset = TensorDataset(features_tensor, labels_tensor)
    data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

    # Calculate class weights
    class_count = y_train[:, emotions.index(emotion)].sum()
    total_count = y_train.shape[0]
    weights = total_count / class_count

    # Loss and Optimizer
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([weights], dtype=torch.float32))
    optimizer = optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-4)

    # Training Loop
    losses = []
    for epoch in tqdm(range(51), desc=f"Training Loop ({emotion})"):
        for features, labels in data_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch {epoch} ({emotion}): Loss: {round(loss.item(),3)}")
            torch.save(model.state_dict(), f'./17-1-25/{emotion}_net_epoch_{epoch}.pth')
            losses.append(round(loss.item(),3))
        if epoch == 50:
            print(f"Epoch {epoch} ({emotion}): Loss: {round(loss.item(),3)}")
            torch.save(model.state_dict(), f'./17-1-25/{emotion}_net_epoch_{epoch}.pth')
            losses.append(round(loss.item(),3))

Training Loop (Joy):   2%|▏         | 1/51 [00:01<01:27,  1.74s/it]

Epoch 0 (Joy): Loss: 0.977


Training Loop (Joy):  22%|██▏       | 11/51 [00:19<01:12,  1.82s/it]

Epoch 10 (Joy): Loss: 1.086


Training Loop (Joy):  41%|████      | 21/51 [00:40<01:01,  2.06s/it]

...

Epoch 30 (Anger): Loss: 0.028


Training Loop (Anger):  80%|████████  | 41/51 [00:42<00:10,  1.01s/it]

Epoch 40 (Anger): Loss: 0.057


Training Loop (Anger): 100%|██████████| 51/51 [00:52<00:00,  1.03s/it]

Epoch 50 (Anger): Loss: 0.017
Epoch 50 (Anger): Loss: 0.017


In [7]:
def get_predictions(X_val, model, threshold=0.5):
    sig = nn.Sigmoid()
    yhat = sig(model(X_val)).detach().numpy()
    y_pred = yhat > threshold
    return y_pred

for emotion in emotions:
    for i in range(5):
        epoch = i*10
        model.load_state_dict(torch.load(f'./17-1-25/{emotion}_net_epoch_{epoch}.pth', weights_only=True))
        y_pred = get_predictions(torch.Tensor(final_val_features[emotion]), model, 0.45)

        val_data_with_pred = pd.DataFrame(y_pred, columns=[emotion])
        val_data_with_pred["id"] = val["id"]

        val_data_with_pred = val_data_with_pred[["id", emotion]]

        current_time = datetime.datetime.now()
        formatted_time = current_time.strftime('%Y-%m-%d_%H_%M_%S')

        val_data_with_pred.to_csv(f'../results/alt4_emolex_1/{emotion}_epoch_{epoch}_pred_eng_a_{formatted_time}.csv', index=False)

        print(val_data_with_pred)

                        id   Joy
0    eng_dev_track_a_00001  True
1    eng_dev_track_a_00002  True
2    eng_dev_track_a_00003  True
3    eng_dev_track_a_00004  True
4    eng_dev_track_a_00005  True
..                     ...   ...
111  eng_dev_track_a_00112  True
112  eng_dev_track_a_00113  True
113  eng_dev_track_a_00114  True
114  eng_dev_track_a_00115  True
115  eng_dev_track_a_00116  True

[116 rows x 2 columns]
                        id   Joy
0    eng_dev_track_a_00001  True
1    eng_dev_track_a_00002  True
2    eng_dev_track_a_00003  True
3    eng_dev_track_a_00004  True
4    eng_dev_track_a_00005  True
..                     ...   ...
111  eng_dev_track_a_00112  True
112  eng_dev_track_a_00113  True
113  eng_dev_track_a_00114  True
114  eng_dev_track_a_00115  True
115  eng_dev_track_a_00116  True

[116 rows x 2 columns]
                        id   Joy
0    eng_dev_track_a_00001  True
1    eng_dev_track_a_00002  True
2    eng_dev_track_a_00003  True
3    eng_dev_track_a_00004  

In [8]:
# from sklearn.metrics import jaccard_score, recall_score, precision_score, f1_score
# from datetime import datetime

# def evaluate(y_true, y_pred):
#     # Calculate Jaccard score
#     jaccard = jaccard_score(y_true, y_pred, average='samples')
#     print(f'Multilabel accuracy (Jaccard score): {round(jaccard, 4)}')
    
#     """Evaluate with micro and macro metrics for multi-label classification"""
#     for average in ['micro', 'macro']:
#         recall = recall_score(y_true, y_pred, average=average, zero_division=0)
#         precision = precision_score(y_true, y_pred, average=average, zero_division=0)
#         f1 = f1_score(y_true, y_pred, average=average, zero_division=0)
    
#         print(f'{average.upper()} recall: {round(recall, 4)}, '
#               f'precision: {round(precision, 4)}, '
#               f'f1: {round(f1, 4)}')

# def evaluate_per_class(y_true, y_pred):
#     """Evaluate metrics for each emotion separately"""
#     for i, emotion in enumerate(emotions):
#         print(f'*** {emotion} ***')
    
#         recall = recall_score(y_true[:,i], y_pred[:,i], zero_division=0)
#         precision = precision_score(y_true[:,i], y_pred[:,i], zero_division=0)
#         f1 = f1_score(y_true[:,i], y_pred[:,i], zero_division=0)
        
#         print(f'recall: {round(recall, 4)}, '
#               f'precision: {round(precision, 4)}, '
#               f'f1: {round(f1, 4)}\n')

# # After getting predictions, add this evaluation code:
# # Evaluate predictions
# print("\nEvaluating validation predictions...")
# val_true = val[emotions].values
# print("Overall Metrics:")
# evaluate(val_true, val_data_with_pred)
# print("\nPer-class Metrics:")
# evaluate_per_class(val_true, val_data_with_pred)

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score, recall_score, precision_score, f1_score
import os

ground_truth = pd.read_csv('../public_data_test/public_data_test/track_a/dev/eng.csv')
y_true = ground_truth[[emotion.lower() for emotion in emotions]].to_numpy()

In [20]:
y_true

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 1, 1, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 1],
       [0, 1, 1, 1, 0],
       [0, 1, 0,

In [21]:
ground_truth

,id,text,anger,fear,joy,sadness,surprise
0,eng_dev_track_a_00001,Older sister (23 at the time) is a Scumbag Stacy.,1,0,0,0,0
1,eng_dev_track_a_00002,"And I laughed like this: garhahagar, because m...",0,1,0,0,0
2,eng_dev_track_a_00003,It overflowed and brown shitty diarrhea water ...,1,1,0,1,1
3,eng_dev_track_a_00004,Its very dark and foggy.,0,1,0,0,0
4,eng_dev_track_a_00005,"Then she tried to, like, have sex with/strangl...",1,1,0,0,1
...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,My heart was beating fast from excitement.,0,0,1,0,0
112,eng_dev_track_a_00113,A fraying rope stretches down from the rafters.,0,1,0,0,1
113,eng_dev_track_a_00114,so i cried my eyes out and did the drawing.,0,0,0,1,0
114,eng_dev_track_a_00115,Never been so close to a group ass-wooping in ...,1,1,0,0,1


In [29]:
for i, emotion in enumerate(emotions):
    # Use glob to find the file matching the pattern
    pattern = os.path.join(results_folder, f"{emotion}_epoch_{epoch}_*.csv")
    matching_files = glob.glob(pattern)

    if matching_files:
        # Use the first matching file
        file_path = matching_files[0]
        emotion_preds = pd.read_csv(file_path)

        # Debugging: Check columns and values
        print(f"Columns: {emotion_preds.columns}")
        print(f"Unique values in {emotion.capitalize()}: {emotion_preds[emotion.capitalize()].unique()}")

        # Handle both 0/1 and True/False cases
        emotion_preds[emotion.capitalize()] = (
            emotion_preds[emotion.capitalize()]
            .astype(str)  # Ensure values are strings for consistency
            .str.strip().str.lower()  # Normalize
        )

        # Map to binary values
        y_pred[:, i] = emotion_preds[emotion.capitalize()].map({
            'true': 1, 'false': 0, '1': 1, '0': 0
        }).fillna(0).astype(int).to_numpy()
    else:
        print(f"No file found for pattern: {pattern}")


Columns: Index(['id', 'Joy'], dtype='object')
Unique values in Joy: [ True False]
Columns: Index(['id', 'Sadness'], dtype='object')
Unique values in Sadness: [ True False]
Columns: Index(['id', 'Surprise'], dtype='object')
Unique values in Surprise: [False  True]
Columns: Index(['id', 'Fear'], dtype='object')
Unique values in Fear: [False  True]
Columns: Index(['id', 'Anger'], dtype='object')
Unique values in Anger: [False  True]


In [32]:
def evaluate(y_true, y_pred):
    jaccard = jaccard_score(y_true, y_pred, average='samples')
    print(f"Jaccard score: {round(jaccard, 4)}")
    
    for average in ['micro', 'macro']:
        recall = recall_score(y_true, y_pred, average=average, zero_division=0)
        precision = precision_score(y_true, y_pred, average=average, zero_division=0)
        f1 = f1_score(y_true, y_pred, average=average, zero_division=0)
        print(f"{average.upper()} - Recall: {round(recall, 4)}, Precision: {round(precision, 4)}, F1: {round(f1, 4)}")

def evaluate_per_class(y_true, y_pred):
    """
    Evaluate precision, recall, and F1 score per class.
    """
    for i, emotion in enumerate(emotions):
        print(f"*** {emotion.capitalize()} ***")
        # Calculate scores for this emotion (label)
        recall = recall_score(y_true[:, i], y_pred[:, i], average=None, zero_division=0)
        precision = precision_score(y_true[:, i], y_pred[:, i], average=None, zero_division=0)
        f1 = f1_score(y_true[:, i], y_pred[:, i], average=None, zero_division=0)

        print(f"  Recall: {recall}")
        print(f"  Precision: {precision}")
        print(f"  F1 Score: {f1}")


In [33]:
# Evaluate each epoch
for epoch, y_pred in predictions_by_epoch.items():
    print(f"--- Epoch {epoch} ---")
    evaluate(y_true, y_pred)
    evaluate_per_class(y_true, y_pred)


--- Epoch 0 ---
Jaccard score: 0.3034
MICRO - Recall: 1.0, Precision: 0.3034, F1: 0.4656
MACRO - Recall: 1.0, Precision: 0.3034, F1: 0.4507
*** Joy ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
*** Sadness ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
*** Surprise ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
*** Fear ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
*** Anger ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
--- Epoch 10 ---
Jaccard score: 0.3034
MICRO - Recall: 1.0, Precision: 0.3034, F1: 0.4656
MACRO - Recall: 1.0, Precision: 0.3034, F1: 0.4507
*** Joy ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
*** Sadness ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
*** Surprise ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
  F1 Score: [0. 0. 0.]
*** Fear ***
  Recall: [0. 0. 0.]
  Precision: [0. 0. 0.]
